In [217]:
import os, sys
import h5py
import numpy as np
from astropy import units
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt

In [2]:
sys.path.append('/Users/belugawhale/Documents/GitHub/wasp39b_paper/scripts/')
from utils import pipeline_dictionary

pipeline_dict = pipeline_dictionary()

In [13]:
def add_contact(header, author, contact):
    head.attrs[u'author'] = u'{}'.format(author)
    head.attrs[u'email'] = u'{}'.format(contact)
    return

## 1.1 Rewriting average traces

In [176]:
outputdir = '/Users/belugawhale/Documents/niriss_real/wasp39/zenodo_upload/1_REDUCTION/1.1_EXAMPLE_TRACES/'
inputdir = './traces'
files = np.sort([i for i in os.listdir(inputdir) if i.endswith('.csv')])

In [191]:
hf.close()
for i in range(len(files)):
    initials = files[i].split('_')[0]
    name = pipeline_dict[initials]['name']
    
    output_name = name + '_order_traces.h5'
    hf = h5py.File(os.path.join(outputdir, output_name), 'w')
    
    head = hf.create_dataset('header',(1,))
    add_contact(head, pipeline_dict[initials]['author'],
                pipeline_dict[initials]['contact'])
    head.attrs[u'units_x'] = u'pixel'
    head.attrs[u'units_order_1'] = u'pixel'
    head.attrs[u'units_order_2'] = u'pixel'
    
    tab = Table.read(os.path.join(inputdir, files[i]), format='csv')
    
    hf.create_dataset('x', data=tab['x'])
    hf.create_dataset('order_1', data=tab['order1'])
    hf.create_dataset('order_2', data=tab['order2'])
    
    try:
        hf.create_dataset('order_3', data=tab['order3'])
        head.attrs[u'units_order_3'] = u'pixel'
    except:
        pass
    
    hf.close()

## 2a Rewriting stellar spectra (nirHiss)

In [203]:
outputdir = '/Users/belugawhale/Documents/niriss_real/wasp39/zenodo_upload/2_STELLAR_SPECTRA/'

In [227]:
files = ['/Users/belugawhale/Downloads/ADF_wasp-39_order1_stellar_spectra_v10.npy',
         '/Users/belugawhale/Downloads/ADF_wasp-39_order2_stellar_spectra_v10.npy']

In [241]:
hf = h5py.File(os.path.join(outputdir, 'nirHiss_stellar_spectra.h5'), 'w')

head = hf.create_dataset('header',(1,))
add_contact(head, pipeline_dict['CMADF']['author'],
            pipeline_dict['CMADF']['contact'])

head.attrs[u'units_time'.format(i+1)] = u'BJD'


for i in range(len(files)):
    data = np.load(files[i], allow_pickle=True)
    
    head.attrs[u'units_wavelength_order_{}'.format(i+1)] = u'micron'
    head.attrs[u'units_flux_order_{}'.format(i+1)] = u'DN/s'
    head.attrs[u'units_flux_err_order_{}'.format(i+1)] = u'DN/s'
    
    if i == 0:
        hf.create_dataset('time', data=d[0]+2400000)
        
    hf.create_dataset('wavelength_order_{}'.format(i+1), data=data[1])
    hf.create_dataset('flux_order_{}'.format(i+1), data=data[2])
    hf.create_dataset('flux_err_order_{}'.format(i+1), data=data[3])

hf.close()

## 2b Rewriting stellar spectra (supreme-SPOON)

In [246]:
files = ['/Users/belugawhale/Downloads/WASP-39_atoca_spectra_fullres.fits',
         '/Users/belugawhale/Downloads/WASP-39_box_spectra_fullres.fits']

In [247]:
method = ['atoca', 'box']

for i in range(len(files)):
    hf = h5py.File(os.path.join(outputdir, 
                                'supreme-SPOON_{}_stellar_spectra.h5'.format(method[i])), 
                   'w')

    
    head = hf.create_dataset('header',(1,))
    add_contact(head, pipeline_dict['MCR']['author'],
                pipeline_dict['MCR']['contact'])

    head.attrs[u'units_time'.format(i+1)] = u'BJD'
    
    hdu = fits.open(files[i])
    
    for n in range(2):
        head.attrs[u'units_wavelength_order_{}'.format(i+1)] = u'micron'
        head.attrs[u'units_flux_order_{}'.format(i+1)] = u'electrons'
        head.attrs[u'units_flux_err_order_{}'.format(i+1)] = u'electrons'
    

    hf.create_dataset('time', data=hdu[9].data)
        
    hf.create_dataset('wavelength_lower_order_1', data=hdu[1].data)
    hf.create_dataset('wavelength_upper_order_1', data=hdu[2].data)
    hf.create_dataset('flux_order_1', data=hdu[3].data)
    hf.create_dataset('flux_err_order_1'.format(i+1), data=hdu[4].data)
    
    hf.create_dataset('wavelength_lower_order_2', data=hdu[5].data)
    hf.create_dataset('wavelength_upper_order_2', data=hdu[6].data)
    hf.create_dataset('flux_order_2', data=hdu[7].data)
    hf.create_dataset('flux_err_order_2'.format(i+1), data=hdu[8].data)

    hf.close()
    
    hdu.close()

## 3.1 Rewriting white light curve files

In [253]:
outputdir = '/Users/belugawhale/Documents/niriss_real/wasp39/zenodo_upload/3_LIGHT_CURVES/3.1_WHITE_LIGHT_CURVES/'
inputdir = './wlcs'

In [254]:
files = np.sort(os.listdir(inputdir))
n = np.array([0, 0, 1, 2, 2, 3, 3, 4, 4, 5])

In [263]:
for i in np.unique(n):
    inds = np.where(n==i)[0]
    initials = files[inds[0]].split('-')[0]
    name = pipeline_dict[initials]['name']
    
    output_name = name + '_white_light_curves.h5'
    hf = h5py.File(os.path.join(outputdir, output_name), 'w')
    
    head = hf.create_dataset('header',(1,))
    add_contact(head, pipeline_dict[initials]['author'],
                pipeline_dict[initials]['contact'])
    head.attrs[u'units_residuals'.format(i+1)] = u'ppm'
    head.attrs[u'units_residuals_err'.format(i+1)] = u'ppm'
    head.attrs[u'units_time'.format(i+1)] = u'time from mid-transit, hours'
    
    for j,fn in enumerate(files[inds]):
        d = np.load(os.path.join(inputdir, fn))
        
        okey = 'order_{}'.format(j+1)
        if j==0:
            hf.create_dataset('time', data=d[0])
        hf.create_dataset('normalized_flux'+okey, data=d[1])
        hf.create_dataset('normalized_flux_err'+okey, data=d[2])
        hf.create_dataset('residuals'+okey, data=d[3])
        hf.create_dataset('residauls_err'+okey, data=d[4])
        hf.create_dataset('best_fit_model'+okey, data=d[5])
    
    hf.close()

## 3.2 Rewriting spectroscopic light curves

In [264]:
outputdir = '/Users/belugawhale/Documents/niriss_real/wasp39/zenodo_upload/3_LIGHT_CURVES/3.2_SPECTROSCOPIC_LIGHT_CURVES/'
inputdir = './spec_lc'

In [307]:
data1 = np.load('../data/spec_lc/NIRISS_WASP-39_ADF_order1_all_models_and_LCs.npy',
               allow_pickle=True).item()
data2 = np.load('../data/spec_lc/NIRISS_WASP-39_ADF_order2_all_models_and_LCs.npy',
               allow_pickle=True).item()

In [314]:
for i in range(2):
    initials = 'CMADF'
    
    data = np.load('../data/spec_lc/NIRISS_WASP-39_ADF_order{}_all_models_and_LCs.npy'.format(i+1),
               allow_pickle=True).item()
    
    if i == 0:
        error_array = np.load('../data/spec_lc/ADF_wasp-39_order1_stellar_spectra_v10.npy',
                              allow_pickle=True)
    else:
        error_array = np.load('../data/spec_lc/ADF_wasp-39_order2_stellar_spectra_v10.npy',
                              allow_pickle=True)

    output_name = 'nirHiss_spectroscopic_light_curves_order_{}.h5'.format(i+1)
    
    hf = h5py.File(os.path.join(outputdir, output_name), 'w')
    
    head = hf.create_dataset('header',(1,))
    add_contact(head, 'Catriona Murray',
                'Catriona.Murray@colorado.edu')
    
    head.attrs[u'units_time'] = u'BJD'
    head.attrs[u'units_wavelength'] = u'micron'
    
    hf.create_dataset('time', data=data['time']+2400000)
    
    for n, j in enumerate(np.arange(1,len(list(data.keys())[1:]),
                                    5,dtype=int)):
        
        wave = data[list(data.keys())[j]]
        
        if wave[0].value > 0.5:
        
            lc = data[list(data.keys())[j+1]]
            model = data[list(data.keys())[j+4]]

            ind = np.where(error_array[1]>=wave[0].value)[0][-1]
            try:
                error = err1[3][:,ind]
                fnorm = err1[2][:,ind]
                errors = (np.sqrt(np.nansum(error))/len(error))/np.nanmax(fnorm)
            except:
                error = 0

            hf.create_dataset('wavelength_channel_{0:04d}'.format(n), 
                              data=wave[0].value)


            hf.create_dataset('light_curve_channel_{0:04d}'.format(n), 
                              data=lc)

            hf.create_dataset('light_curve_error_channel_{0:04d}'.format(n),
                              data=np.full(len(lc), errors))


            hf.create_dataset('best_fit_model_channel_{0:04d}'.format(n),
                              data=model)

    hf.close()

## 4 Rewriting transmission spectra

In [326]:
outputdir = '/Users/belugawhale/Documents/niriss_real/wasp39/zenodo_upload/4_TRANSMISSION_SPECTRA/'
inputdir = './ts'

In [327]:
def write_file(hf, data, order, author, email):
    
    head = hf.create_dataset('header',(1,))
    head.attrs[u'units_wavelength'] = u'micron'
    head.attrs[u'units_wavelength_err'] = u'micron'
    head.attrs[u'units_transit_depth'] = u'ppm'
    head.attrs[u'units_transit_depth_err'] = u'ppm'
    
    add_contact(head, author, email)
    
    hf.create_dataset('wavelength_order_{}'.format(order), 
                      data=data['wave'])
    hf.create_dataset('wavelength_err_order_{}'.format(order),
                      data=data['wave_error'])
    hf.create_dataset('transit_depth_order_{}'.format(order),
                      data=data['dppm'])
    hf.create_dataset('transit_depth_err_order_{}'.format(order),
                      data=data['dppm_err'])
    hf.create_dataset('quality_flags_order_{}'.format(order),
                      data=data['quality'])
    return 

In [328]:
files = np.sort(os.listdir(inputdir))
n = np.array([0, 1, 1,1,1, 2, 2, 3, 3, 4, 4, -1, 6, 6])

for i in np.unique(n):
    if i >= 0:
        inds = np.where(n==i)[0]
        initials = files[inds[0]].split('-')[0]
        name = pipeline_dict[initials]['name']
        
        for j in range(len(inds)): 
            
            if len(inds) == 1:
                R = 300
            else:
                close = files[inds[j]].split('_')[-1][:-4]
                if '100' in close:
                    R = 100
                elif '300' in close:
                    R = 300
                elif 'binned2' in close:
                    R = 'native'
                elif 'highres' in close:
                    R = 'pixel'
                else:
                    R = 300
            
            output_name = name + '_transmission_spectrum_R_{}.h5'.format(R)
            
            try:
                hf = h5py.File(os.path.join(outputdir, 
                                        output_name), 'w')
                
                data = Table.read(os.path.join(inputdir, files[inds[j]]), 
                                  format='csv', comment='#')

                for order in [1,2]:
                    write_file(hf, data[data['order']==order], order,
                               pipeline_dict[initials]['author'],
                               pipeline_dict[initials]['contact'])

                hf.close()                
            except:
                pass

## 5.1 Rewriting grid fit models

In [329]:
outputdir = '/Users/belugawhale/Documents/niriss_real/wasp39/zenodo_upload/5_THEORY/5.1_BESTFIT_GRID_MODELS/'
inputdir = './grid_best/'
files = np.sort(os.listdir(inputdir))

In [330]:
for fn in files:
    print(fn)
    
    data = np.loadtxt(os.path.join(inputdir, fn))
    output_name = fn[:-5] + '.h5'
    
    hf = h5py.File(os.path.join(outputdir, output_name), 'w')
    head = hf.create_dataset('header',(1,))
    
    add_contact(head, 'Kazumasa Ohno', 'kono2@ucsc.edu')
    head.attrs[u'units_wavelength'] = u'micron'
    head.attrs[u'units_transit_depth'] = u'ppm'
    

    with open(os.path.join(inputdir, fn), encoding='utf-8') as inf:
        f = inf.readlines()

    parameters = f[1][1:-2].split(',')

    for i in range(len(parameters)):
        params = parameters[i].split('=')
        key = params[0].replace(' ', '')
        val = np.float(params[1])
        head.attrs[u'{}'.format(key)] = val
        
    hf.create_dataset('wavelength'.format(order), 
                      data=data[:,0])
    hf.create_dataset('transit_depth'.format(order),
                      data=data[:,1])
        
    hf.close()

ATMO_clear_R300.spec
ATMO_cloudy_R300.spec
PICASO_clear_R300.spec
PICASO_cloudy_R300.spec
Phoenix_clear_R300.spec
Phoenix_cloudy_R300.spec


/var/folders/9s/jb487nv52hs8n0q5ftmb1lk80000gn/T/ipykernel_77422/1515062228.py:23: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  val = np.float(params[1])


## 5.2 Rewriting contribution plot data

In [331]:
outputdir = '/Users/belugawhale/Documents/niriss_real/wasp39/zenodo_upload/5_THEORY/5.2_CONTRIBUTION_MODELS/'
inputdir = './contributions/models/'
files = np.sort([i for i in os.listdir(inputdir) if i.endswith('.txt')])[1:]

In [332]:
for fn in files:
    
    data = np.loadtxt(os.path.join(inputdir, fn))
    
    split = fn.split('x')
    output_name = 'model_without_'+split[1][:-2]+'.h5'
    
    hf = h5py.File(os.path.join(outputdir, output_name), 'w')
    
    head = hf.create_dataset('header',(1,))
    add_contact(head, 'Luis Welbanks', 'luis.welbanks@asu.edu')
    
    head.attrs[u'units_wavelength'] = u'micron'
    head.attrs[u'units_transit_depth'] = u'ppm'
    
    hf.create_dataset('wavelength'.format(order), 
                      data=data[:,0])
    hf.create_dataset('transit_depth'.format(order),
                      data=data[:,1])
    
    hf.close()

#### Rewriting cross-sections

In [334]:
table = Table.read('/Users/belugawhale/Documents/GitHub/wasp39b_paper/data/contributions/xsecs/xsections.txt',
                   format='csv', delimiter=' ')

In [160]:
hf = h5py.File(os.path.join(outputdir, 'species_cross_sections.h5'), 'w')
    
head = hf.create_dataset('header',(1,))
add_contact(head, 'Luis Welbanks', 'luis.welbanks@asu.edu')

head.attrs[u'units_wavelength'] = u'micron'
head.attrs[u'units_xsection'] = u'm^2'

hf.create_dataset('wavelength'.format(order), data=table['wavelength'])

for col in table.colnames[1:]:
    hf.create_dataset(col[1:], data=table[col])
    
hf.close()

## 5.4 Rewriting potassium models

In [342]:
outputdir = '/Users/belugawhale/Documents/niriss_real/wasp39/zenodo_upload/5_THEORY/5.4_RULEOUT_K_to_O_MODELS/'
inputdir = './ko_short_lam/'
files = np.sort([i for i in os.listdir(inputdir) if i.endswith('.txt')])[1:]

In [347]:
hf = h5py.File(os.path.join(outputdir, 'ScChimera_K_to_O_models.h5'), 'w')
    
head = hf.create_dataset('header',(1,))
add_contact(head, 'Luis Welbanks', 'luis.welbanks@asu.edu')

head.attrs[u'units_wavelength'] = u'micron'
head.attrs[u'units_transit_depth'] = u'ppm'

for i in range(len(files)):
    
    data = np.loadtxt(os.path.join(inputdir, files[i]))
    
    val = float(files[i].split('_')[0])
    
    head.attrs[u'[K/O]_{}'.format(i)] = val
    data = np.loadtxt(os.path.join(inputdir, files[i]))
    
    hf.create_dataset('model_{}_transit_depth'.format(i), data=data[:,1])
    
hf.create_dataset('wavelength', data=data[:,0])
    
hf.close()

## 5.5 Rewriting rule-out C/O models

In [350]:
outputdir = '/Users/belugawhale/Documents/niriss_real/wasp39/zenodo_upload/5_THEORY/5.5_RULEOUT_C_to_O_MODELS/'
inputdir = './model_rule_out_co/'
files = np.sort([i for i in os.listdir(inputdir) if i.endswith('.txt')])
co_vals = [0.55, 0.70, 0.80]

In [351]:
hf = h5py.File(os.path.join(outputdir, 'ScChimera_C_to_O_models.h5'), 'w')
    
head = hf.create_dataset('header',(1,))
add_contact(head, 'Luis Welbanks', 'luis.welbanks@asu.edu')

head.attrs[u'units_wavelength'] = u'micron'
head.attrs[u'units_transit_depth'] = u'ppm'

for i in range(len(co_vals)):
    head.attrs[u'C/O_{}'.format(i)] = co_vals[i]

    data = np.loadtxt(os.path.join(inputdir, files[i]))
    
    hf.create_dataset('model_{}_transit_depth'.format(i), data=data[:,1])

hf.create_dataset('wavelength', data=data[:,0])

hf.close()

## 5.6 Rewriting rule-out metallicity models

In [352]:
outputdir = '/Users/belugawhale/Documents/niriss_real/wasp39/zenodo_upload/5_THEORY/5.6_RULEOUT_METALLICITY_MODELS/'
inputdir = './model_rule_out_z/'
files = np.sort([i for i in os.listdir(inputdir) if i.endswith('.txt')])
z_vals = [0.0, 0.5, 1.0, 1.5, 2.0, 2.25]

In [354]:
hf = h5py.File(os.path.join(outputdir, 'ScChimera_metallicity_models.h5'), 'w')
    
head = hf.create_dataset('header',(1,))
add_contact(head, 'Luis Welbanks', 'luis.welbanks@asu.edu')

head.attrs[u'units_wavelength'] = u'micron'
head.attrs[u'units_transit_depth'] = u'ppm'

for i in range(len(z_vals)):
    head.attrs[u'[M/H]_{}'.format(i)] = z_vals[i]
    data = np.loadtxt(os.path.join(inputdir, files[i]))
    
    hf.create_dataset('model_{}_transit_depth'.format(i), data=data[:,1])

hf.create_dataset('wavelength', data=data[:,0])

hf.close()